In [1]:
import pandas as pd
import pyspark
import os 
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf      # sf = spark functions
import pyspark.sql.types as st          # st = spark types

spark = SparkSession.builder.master('local').getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/02/14 11:01:41 WARN Utils: Your hostname, DESKTOP-SU1U0NI resolves to a loopback address: 127.0.1.1; using 172.21.244.112 instead (on interface eth0)
23/02/14 11:01:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/14 11:01:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data_dir = './data'

cpi_file = 'US-CPI.csv'
unemp_file = 'USUnemployment.csv'

c_schema = 'Yearmon date, CPI float'
u_schema = 'Year date, Jan int ,Feb int ,Mar int,Apr int ,May int,Jun int ,Jul int ,Aug int ,Sep int,Oct int,Nov int,Dec int'

cpi_df = spark.read.csv(os.path.join(data_dir, cpi_file), schema=c_schema, header=0, enforceSchema=True)
unemp_df = spark.read.csv(os.path.join(data_dir, unemp_file), schema=u_schema, header=0, enforceSchema=True)

cpi_df.printSchema()
unemp_df.printSchema()

root
 |-- Yearmon: date (nullable = true)
 |-- CPI: float (nullable = true)

root
 |-- Year: date (nullable = true)
 |-- Jan: integer (nullable = true)
 |-- Feb: integer (nullable = true)
 |-- Mar: integer (nullable = true)
 |-- Apr: integer (nullable = true)
 |-- May: integer (nullable = true)
 |-- Jun: integer (nullable = true)
 |-- Jul: integer (nullable = true)
 |-- Aug: integer (nullable = true)
 |-- Sep: integer (nullable = true)
 |-- Oct: integer (nullable = true)
 |-- Nov: integer (nullable = true)
 |-- Dec: integer (nullable = true)



In [ ]:
# rename cols in both files

def rename_cols():
  for column in df.columns: 
    df = df.withColumnRenamed(column, column.lower())
  return df

udf_rename = sf.udf(rename_cols)

cpi_df = cpi_df.apply(udf_rename)
unemp_df = unemp_df.apply(udf_rename)

cpi_df.show(5)
unemp_df.show(5)

# # calculate avg unemployment rate per year
# unemp_df = unemp_df.withColumn("avg_unemp_rate", sf.avg.array('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'))

# # calculate avg inflation rate per year
# cpi_df = cpi_df.groupBy('yearmon').withColumn('avg_CPI', avg())